In [46]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException

from datetime import datetime
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import datetime
import io
import platform
from PyToSp import *
from TrackingFlatFile import *
from GetProjectKey import *
from utils import *
from tqdm import tqdm


In [8]:
import datetime
import pytz
my_date = datetime.datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%Y-%m-%d %H:%M:%S.%f")
my_date = datetime.datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%Y-%m-%d %H:%M:%S.%f")
my_date = datetime.datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%Y-%m-%d %H:%M:%S.%f")
my_date = datetime.datetime.now(pytz.timezone('Asia/Bangkok')).strftime("%Y-%m-%d %H:%M:%S.%f")

'2021-10-28 15:28:35.259328'

# Connect to SQL azure database

In [47]:
## Connect to SQL azure database
# Credential

server='hkazdevsqld3vnreserch.database.windows.net'
database='D3VNResearch_Staging'
username =  'D3VNResearch@savills.com.vn'
password = '@Advisory092021!'
auth = 'ActiveDirectoryPassword'
#Connect SQL
cnxn=pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER="+server+";DATABASE="+database+";UID="+username+";PWD="+password+";Authentication="+auth)

# Connect to SharePoint

In [26]:
# Get authentication from json config file 

header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub) # config_path the same path with PyToSp module

header_VNBIHNS = 'share_point_VNBIHNS'
config_VNBIHNS = read_config_json(config_path, header_VNBIHNS) # config_path the same path with PyToSp module
header_VNBIHCM='share_point_VNBIHCM'
config_VNBIHCM=read_config_json(config_path, header_VNBIHCM)

BIHub = SharePoint(config_BIHub)
VNBIHNS = SharePoint(config_VNBIHNS)
VNBIHCM=SharePoint(config_VNBIHCM)
VNBIHCM.check_connect()

Web site title: VN, BI - HCMC RS _ Successful Connection!


# Run Audit Tracking

In [27]:
## Processing Folder
relative_url = "/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3"
df_summ_file = VNBIHNS.get_content_url(relative_url)
list_file = df_summ_file['ServerRelativeUrl'].to_list()
df_query=pd.DataFrame(df_summ_file)
data=pd.read_sql('select * from GENERAL.City_Dictionary',cnxn)
Raw_City=dict(zip(data['Raw_City'],data['Cleaned_City']))


Files name: /sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/SA_HN_20210930.csv
Files name: /sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Retail_HN_20210901_20211012.csv
Files name: /sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Office_HN_20210930_20211014.csv
Files name: /sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Retail_HN_20210901_20211012.xlsx
Files name: /sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Apartment_HN_20210930_20211012.csv
Files name: /sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Hotel_HN_20210930_20211014.csv
Files name: /sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/VLTH_HN_20210930.csv


In [16]:
def Create_Audit(cnxn,Raw_City,list_file,df_query,obj_sp):
    for file_url in list_file:
        if file_url.split('/')[-1] in ['Retail_HN_20210901_20211012.csv']:
            print("pass")
            pass
        else:
            crsr = cnxn.cursor()
            summ(obj_sp, df_query, file_url,crsr,Raw_City)
    cnxn.commit()
    crsr.close()
Create_Audit(cnxn,Raw_City,list_file,df_query,VNBIHNS)

pass
/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Office_HN_20210930_20211014.csv
/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Retail_HN_20210901_20211012.xlsx
/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Apartment_HN_20210930_20211012.csv
/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/Hotel_HN_20210930_20211014.csv
/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/SA_HN_20210930_20211012.xlsx
/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/VLTH_HN_20210901_20211014.csv


In [7]:
#data=VNBIHCM.get_file('/sites/VN-BI-HCMCRS/Shared Documents/HCMC RS/Flat input file/1. Main Sectors/2021/Q3/Office_HCMC_20210930.csv')
#data=data.replace(r'\* \ *',np.nan)
#data['retail.OCCUPANCY']=data['retail.OCCUPANCY'].astype(float)
#data['retail.OCCUPANCY'][0:50]
#data.loc[:,'retail.OCCUPANCY'].sum()
#pd.to_datetime(data['office.DATE_KEY'].iloc[0],format='%Y%m%d')

Timestamp('2021-09-30 00:00:00')

# Get Project Key

In [48]:
relative_url_upload = '/sites/BIHub/Shared Documents/Test'
sql_query = "SELECT * FROM GENERAL.City_Dictionary"
df_city_dict = pd.read_sql(sql_query,cnxn)
save_to = '/Users/khoi/Desktop/test/'
for file_url in tqdm(list_file):
    if file_url.split('/')[-1] == 'Retail_HN_20210901_20211012.csv':
        pass
    elif file_url.split('/')[-1] == 'VLTH_HN_20210930.csv':
#         Total_City=pd.read_sql('select * from GENERAL.City_Dictionary',cnxn)
        # for file in csv_files:
        print(file_url.split('/')[-1])
        cursor=cnxn.cursor()
        project_name=file_url.split('/')[-1]
        format_file=project_name.split('.')[1]
        Sector=file_url.split('/')[-1].upper().split('_')[0]
        if Sector=='SA':
            Sector='SERVICED_APARTMENT'
        elif Sector=='APT':
            Sector='APARTMENT'
        Raw_City=file_url.split('/')[-1].upper().split('_')[1]
        
        ''' Modify Object Sharepoint here'''
        data= VNBIHNS.get_file(file_url)
        data=data.dropna(how='all')
        data.columns=data.columns.str.strip()
        if 'LATITUDE' and 'LONGITUDE' in data.columns:
            data=data.rename(columns = {'LATITUDE':'LAT'})
            data=data.rename(columns = {'LONGITUDE':'LONG'})
        for i in list(data.columns):
            data[i] = data[i].replace({np.nan: None})
        data['location.CITY_NAME_DOC']=data[['location.CITY_NAME_DOC']].merge(df_city_dict,how='left',left_on=['location.CITY_NAME_DOC'], right_on=['Raw_City'])['Cleaned_City']
        City = str(data['location.CITY_NAME_DOC'][1])
        project_key=Project_key(City,Sector,cnxn)
        data=Convert_District(data,cnxn)
        if Sector=='VLTH' or Sector=='RETAIL':
            project_key['SUB_PROJECT_TYPE']=project_key['SUB_PROJECT_TYPE'].str.strip()
            data['project.SUB_PROJECT_TYPE']=data['project.SUB_PROJECT_TYPE'].str.strip()
        #Xử lí LAT, LONG bị dấu (,)
        data=Convert_Location(data,cnxn)
        break
        #Điền các values từ Project_Name nếu Sub_Project_Name NULL
        data['project.ORIGINAL_SUB_PROJECT_NAME_DOC']=data['project.ORIGINAL_SUB_PROJECT_NAME_DOC'].fillna(data['project.PROJECT_NAME_DOC'])
        data['project.PROJECT_NAME_DOC']=data['project.PROJECT_NAME_DOC'].fillna('project.ORIGINAL_SUB_PROJECT_NAME_DOC')
        ##Loại bỏ space ở word
        data['project.ORIGINAL_SUB_PROJECT_NAME_DOC']=data['project.ORIGINAL_SUB_PROJECT_NAME_DOC'].astype(str).str.strip()
        #Left join 2 Dataframe
        Merge_data=Merge(Sector,data,project_key)
        #Lọc các Projectkey NULL
        empty=Merge_data[Merge_data['PROJECT_KEY'].isna()]
        #Convert LAUNCHING_TIME
        empty=Convert_time(empty)
        if empty.shape[0]!=0:
#             Insert_DB(Sector,empty,cnxn,City)
            Project_key_new=Project_key(City,Sector,cnxn)
            result=Merge(Sector,data,Project_key_new)
            data['project.PROJECT_KEY']=result['PROJECT_KEY']
            project_name.split('.')[1]
            data = data_cleaning(data, Sector)
            # Insert data to sql database
            insert_to_fact(data, Sector, cnxn)
            
            # Update file to sharepoint
            filename = '{}.csv'.format(project_name.split('.')[0])
            BIHub.upload_dataframe(relative_url_upload, data, filename)
        else:
            data['project.PROJECT_KEY']=Merge_data['PROJECT_KEY']
            data = data_cleaning(data, Sector)
            # Insert data to sql database
            insert_to_fact(data, Sector, cnxn)
            
            # Update file to sharepoint
            filename = '{}.csv'.format(project_name.split('.')[0])
            BIHub.upload_dataframe(relative_url_upload, data, filename)





  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

VLTH_HN_20210930.csv
Read csv with Encoding = cp1252 ...


 86%|████████████████████████████████████████████████████████████████████████            | 6/7 [00:00<00:00,  8.50it/s]


In [44]:
data=VNBIHNS.get_file('/sites/VN-BI-HNRS/Shared Documents/HN RS/Flat input file/1. Main Sectors/2021/Q3/VLTH_HN_20210930.csv')
data['LAT']

Read csv with Encoding = cp1252 ...


0      21.019289
1      21.018501
2      21.018501
3      21.028996
4      21.028996
         ...    
503          NaN
504          NaN
505          NaN
506          NaN
507          NaN
Name: LAT, Length: 508, dtype: float64

# Upload to SharePoint

In [11]:
# from io import BytesIO
relative_url_upload = '/sites/BIHub/Shared Documents/Test'
filename = 'Office_HN_20210930_20211014.csv'
BIHub.upload_dataframe(relative_url_upload, data, filename)

TypeError: a bytes-like object is required, not 'str'